In [1]:
import os
from pymongo import MongoClient
import pandas as pd
import json
import requests
from dotenv import load_dotenv
import re
load_dotenv()

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


True

In [2]:
startmongo = 'service mongod start'
command = 'mongoimport --db companies --collection companies --file Database/companies.json'
def initDB():
    os.popen(startmongo)
os.popen(startmongo)
os.popen('service mongod status')
os.popen(command)

In [3]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [4]:
db,coll = (list(connectCollection('companies','companies')))
rest = (list(coll.find({'$and':[{'$or':[{"offices.latitude":{'$ne':None}},{"offices.longitude":{'$ne':None}}]},{'deadpooled_year':None}]})))
value =  rest[0].get('total_money_raised')

In [5]:
#q = {'$or':[{"offices.latitude":{'$ne':'null'}},{"offices.longitude":{'$ne':'null'}}]}
rest = (list(coll.find({'$and':[{'$or':[{"offices.latitude":{'$ne':None}},{"offices.longitude":{'$ne':None}}]},{'deadpooled_year':None}]})))
clean_db = []
for i in range(len(rest)):
    value = rest[i].get('total_money_raised') 
    valueM = re.findall(r'\d+[?\.\d]?\d',value)
    if valueM == []:
        valueM = [0]
    diction ={
        'name':rest[i]['name'],
        'category':rest[i]['category_code'],
        #'deadpooled':rest[i]['deadpooled_year'],
        'value': {
            'value':float(valueM[0]),
            'qty':value[-1]
        }
        
    }
    for ind in range(len(rest[i]['offices'])):
        if rest[i]['offices']:
            diction[f'Location-{ind+1}'] = {
                'type':'Point',
                'coordinates':[float(rest[i]['offices'][ind]['longitude']), float(rest[i]['offices'][ind]['latitude'])]
            }
        
    
    clean_db.append(diction)

In [6]:
with open('Database/database_clean.json', 'w') as fp:
    json.dump(clean_db, fp)

In [7]:
#command = 'mongoimport --db companies --collection companies_clean --jsonArray --file database_clean.json'
#os.popen(command)

'''def requestfoursquare(query,lon,lat,radius):
    '''
    #lon => float
    #lat => float
    #radius => meters
    '''
    url = 'https://api.foursquare.com/v2/venues/explore'
    #client = os.getenv("CLIENT_ID")
    #passw = os.getenv("CLIENT_SECRET")
    params = dict(
        client_id=os.getenv("CLIENT_ID"),
        client_secret=os.getenv("CLIENT_SECRET"),
        v='20180335',
        ll='{},{}'.format(lat,lon),
        query=query,
        limit=200,
        radius = radius
    )
    resp = requests.get(url=url, params=params)
    data = json.loads(resp.text)
    return data'''
    
'''def getlocation(data):
    locationslist = []
    for i in range(len(data['response']['groups'][0]['items'])):
        latitude = float((data['response']['groups'][0]['items'][i]['venue'].get('location')).get('lat'))
        longitude = float((data['response']['groups'][0]['items'][i]['venue'].get('location')).get('lng'))
        dictio = {
            'name':(data['response']['groups'][0]['items'][i]['venue']).get('name'),
            'loc':{'type':'Point','coordinates':[longitude,latitude]}            
            }
        locationslist.append(dictio)
    return locationslist
'''

'''#'name: Starbucks'
for i in range(len(clean_db)):
    for ind in range(1,(len(clean_db[i])-2)):
        print('enterprise{}, location{}'.format(i,ind))
        lat = clean_db[i]['Location-{}'.format(ind)]['coordinates'][0]
        lon = clean_db[i]['Location-{}'.format(ind)]['coordinates'][1]
        data = requestfoursquare('type: Starbucks',lat,lon,1000)
        display(data)
        geo = getlocation(data)
        print(len(geo))
        break'''


In [69]:
load_dotenv()
db,coll_clean = (list(connectCollection('companies','companies_clean')))

In [70]:
def gsearch(lat,lon,radius,query):
    URL = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
    params={
        'name':str(query),
        'location':'{},{}'.format(lat,lon),
        'radius':str(radius),
        'key':os.getenv('GOOGLE_KEY'),
        }
    res=requests.get(URL, params=params).json()
    return res

In [85]:
def geolocation(data):
    list_places = []
    for i in range(len(data['results'])):
        diction={}
        lat = data['results'][0]['geometry']['location']['lat']
        lng = data['results'][0]['geometry']['location']['lng']
        diction = {
            'type':'Point',
            'coordinates':[lng,lat]
            }
        list_places.append(diction)
    return list_places

In [128]:
def getnearbyplaces(coll,radius,query):
    coll_list = (list(coll.find({})))
    for i in range(len(coll_list)):
        print(len(coll_list[i]))
        print(range(1,(len(coll_list[i])-3)))
        for ind in range(1,(len(coll_list[i])-3)):
            print(ind)
            collitem =  coll_list[i][f'Location-{ind}']
            print('enterprise{}, location{}'.format(i,ind))
            lat = collitem['coordinates'][1]
            lon = collitem['coordinates'][0]
            data = gsearch(lat,lon,radius,query)
            geoloc = geolocation(data)
            locations = geoloc
            places = len(geoloc)
            value = {"$set":{f'Location-{ind}':{'type':'Point','coordinates':[float(lat), float(lon)],query:{'locations':locations,'Nº':places}}}}
            coll.update_one(coll_list[i],value)
            display(coll_list[i])

In [ ]:
getnearbyplaces(coll_clean,500,'starbucks')

In [ ]:
def updatedb(data,collitem,query):
    locations = data
    places = len(data)
    value = {"$set": {f'locations-{query}':data, f'Nº{query}':places}}
    coll.update_one(collitem,value)
updatedb()
